In [2]:
import firebase_admin # Note must run: pip install --no-binary :all: grpcio --ignore-installed
from firebase_admin import credentials
from firebase_admin import firestore

import json
from datetime import datetime

In [3]:
# Use service account
cred = credentials.Certificate('./serviceAccountKey.json')
firebase_admin.initialize_app(cred)

# Initilize db client
db = firestore.client()

In [4]:
with open('./data/firestore_permits.geojson', 'r') as f:
    permits = json.load(f)['features']

In [5]:
def groupby_eventid(permits: list[dict]) -> list:
    events = []
    for perm in permits:
        props = perm['properties']
        event = {
            u'eventid': f'{props["eventid"]}',
            u'startdate': datetime.strptime(props['startdate'], '%Y-%m-%d'),
            u'enddate': datetime.strptime(props['enddate'], '%Y-%m-%d'),
            u'boro': props['boro'].capitalize(),
            u'category': props['category'],
            u'subcategory': props['subcategory'],
            u'country': props['country'],
            u'streets': props['parkingheld'],
            u'geoms': {
                u'type': 'FeatureCollection',
                u'features': [{
                    u'type': 'feature',
                    u'properties': {'eventid': props['eventid']},
                    u'geometry': f"{perm['geometry']}"
                }]
            }
        }
        events.append(event)

    return events

In [6]:
docs = groupby_eventid(permits)

In [9]:
ref = db.collection('permits')
query = ref.where('country', '==', 'Japan').limit(1)
results = query.get()

In [12]:
results[0].to_dict()

{'eventid': '43237',
 'country': 'Japan',
 'enddate': DatetimeWithNanoseconds(2012, 1, 5, 0, 0, tzinfo=datetime.timezone.utc),
 'boro': 'Manhattan',
 'category': 'Television',
 'subcategory': 'News',
 'startdate': DatetimeWithNanoseconds(2012, 1, 5, 0, 0, tzinfo=datetime.timezone.utc),
 'streets': 'mulberry street between spring street and kenmare street',
 'geoms': {'type': 'FeatureCollection',
  'features': [{'geometry': "{'type': 'LineString', 'coordinates': [[-73.99657460431074, 40.721173841340175], [-73.9965408, 40.7212591], [-73.9962831, 40.7219082], [-73.99626569557012, 40.72195215861719]]}",
    'properties': {'eventid': 43237},
    'type': 'feature'}]}}

In [27]:
def write_metadata(docs: list, db: firestore.client):
    meta_ref = db.collection('metadata').document('metadata')
    result = meta_ref.get().to_dict()

    latest = result['latest_start']
    earliest = result['earliest_start']
    data_updates = result['data_updates'] + [datetime.now()]
    countries = result['countries']
    categories = result['categories']
    new_latest = None
    new_earliest = None

    for doc in docs:
        if new_latest == None:
            new_latest = doc['startdate']
        if new_earliest == None:
            new_earliest = doc['startdate']
        if doc['startdate'] > new_latest:
            new_latest = doc['startdate']
        if doc['startdate'] < new_earliest:
            new_earliest = doc['startdate']
        if doc['country'] not in countries:
            countries.append(doc['country'])
        if doc['category'] not in categories.keys():
            categories[doc['category']] = []
        if doc['subcategory'] not in categories[doc['category']]:
            categories[doc['category']].append(doc['subcategory'])

    metadata = {
        'latest_start': latest + [new_latest],
        'earliest_start': earliest + [new_earliest],
        'data_updates': data_updates,
        'countries': countries,
        'categories': categories,
        'update_count': firestore.Increment(1)
    }

    meta_result = meta_ref.update(metadata)

    return meta_result

In [15]:
def write_permits(docs: list, db: firestore.client):    
    # Set batch operation
    batch = db.batch()

    # Batch write to Firestore
    results = []
    for i in range(0, len(docs)):
        doc = docs[i]
        ref = db.collection(u'permits').document(doc['eventid'])
        batch.set(ref, doc)
        if i % 500 == 0:
            result = batch.commit()
            results.append(result)
    result = batch.commit()
    results.append(result)

    # Write metadata
    meta_result = write_metadata(docs, db)

    return results, meta_result

In [16]:
results, meta_result = write_permits(docs, db)

In [19]:
meta_result

update_time {
  seconds: 1652364859
  nanos: 894707000
}